In [1]:
import pandas as pd
import pymorphy2
import numpy as np
import re
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder

ctb = CatBoostClassifier(random_seed=17, iterations=1000, loss_function='CrossEntropy', eval_metric='AUC')                                   
morph = pymorphy2.MorphAnalyzer()

train = pd.read_csv(r'''C:\Users\filip\Downloads\train.csv''')
test = pd.read_csv(r'''C:\Users\filip\Downloads\test.csv''')

In [2]:
def normalize(df):
    df['Upper'] = df['Word'].apply(lambda x: 1 if (x[0].isupper() and x[1:].islower()) else 0)
    df['Lower'] = df['Word'].apply(lambda x: 1 if (x.islower()) else 0)
    df['lenght'] = df['Word'].apply(lambda x: len(x))
    df['All_upper'] = df['Word'].apply(lambda x: 1 if (x.isupper()) else 0)
    df['Vow'] = df['Word'].apply(lambda x: len(re.findall('[уеыаоэяию]', x, re.IGNORECASE)))
    #df['Con'] = df['length'] - train['Vow']
    df['2end'] = df['Word'].str[-2:].str.lower()
    
    
    #df['Word'] = vect_char.fit_transform(df['Word'])
    
    
    df['pymorphy'] = df['Word'].apply(lambda x: morph.tag(x)[0])

    df['pymorphy_animacy'] = df['pymorphy'].apply(lambda x: x.animacy)
    df['pymorphy_POS'] = df['pymorphy'].apply(lambda x: x.POS)
    df['pymorphy_case'] = df['pymorphy'].apply(lambda x: x.case)
    df['pymorphy_number'] = df['pymorphy'].apply(lambda x: x.number)
    df['pymorphy_gender'] = df['pymorphy'].apply(lambda x: x.gender)

    df.drop('pymorphy' , axis=1 , inplace=True)

    columns_to_one_hot = ['pymorphy_animacy', 'pymorphy_POS', 'pymorphy_case','pymorphy_number', 'pymorphy_gender']

    for col in columns_to_one_hot:
        df[col] = LabelEncoder().fit_transform(list(df[col].fillna('nan')))
    
    df['Word'] = df['Word'].apply(lambda x: morph.parse(x)[0].normal_form)
    df['2end_norm'] = df['Word'].str[-2:].str.lower()
    df.drop('Word', axis=1, inplace=True)
    
    return df

n_train = normalize(train)
n_test = normalize(test)

In [3]:
train_col= n_train.columns.tolist()
train_col.remove('Label')
cat_features = np.where(n_train[train_col].dtypes == 'object')[0].tolist()

In [4]:
ctb = CatBoostClassifier(random_seed=17, iterations=1000, loss_function='CrossEntropy', eval_metric='AUC')
ctb.fit(n_train[train_col], n_train['Label'], cat_features=cat_features)

0:	learn: 0.8406619	total: 159ms	remaining: 2m 39s
1:	learn: 0.8467779	total: 233ms	remaining: 1m 56s
2:	learn: 0.8474261	total: 323ms	remaining: 1m 47s
3:	learn: 0.8487522	total: 425ms	remaining: 1m 45s
4:	learn: 0.8482476	total: 496ms	remaining: 1m 38s
5:	learn: 0.8553711	total: 566ms	remaining: 1m 33s
6:	learn: 0.8548770	total: 641ms	remaining: 1m 30s
7:	learn: 0.8543263	total: 713ms	remaining: 1m 28s
8:	learn: 0.8537405	total: 795ms	remaining: 1m 27s
9:	learn: 0.8556943	total: 873ms	remaining: 1m 26s
10:	learn: 0.8564926	total: 960ms	remaining: 1m 26s
11:	learn: 0.8559693	total: 1.03s	remaining: 1m 24s
12:	learn: 0.8559288	total: 1.11s	remaining: 1m 24s
13:	learn: 0.8559453	total: 1.2s	remaining: 1m 24s
14:	learn: 0.8618163	total: 1.3s	remaining: 1m 25s
15:	learn: 0.8674406	total: 1.38s	remaining: 1m 25s
16:	learn: 0.8702445	total: 1.45s	remaining: 1m 24s
17:	learn: 0.8736536	total: 1.53s	remaining: 1m 23s
18:	learn: 0.8763506	total: 1.6s	remaining: 1m 22s
19:	learn: 0.8778883	tota

159:	learn: 0.9116476	total: 13s	remaining: 1m 8s
160:	learn: 0.9118481	total: 13.1s	remaining: 1m 8s
161:	learn: 0.9118986	total: 13.2s	remaining: 1m 8s
162:	learn: 0.9119334	total: 13.3s	remaining: 1m 8s
163:	learn: 0.9119837	total: 13.4s	remaining: 1m 8s
164:	learn: 0.9120002	total: 13.5s	remaining: 1m 8s
165:	learn: 0.9120192	total: 13.6s	remaining: 1m 8s
166:	learn: 0.9120904	total: 13.7s	remaining: 1m 8s
167:	learn: 0.9121151	total: 13.8s	remaining: 1m 8s
168:	learn: 0.9121289	total: 13.8s	remaining: 1m 8s
169:	learn: 0.9121751	total: 13.9s	remaining: 1m 7s
170:	learn: 0.9122747	total: 14s	remaining: 1m 7s
171:	learn: 0.9122982	total: 14.1s	remaining: 1m 7s
172:	learn: 0.9123123	total: 14.2s	remaining: 1m 7s
173:	learn: 0.9123271	total: 14.2s	remaining: 1m 7s
174:	learn: 0.9123548	total: 14.3s	remaining: 1m 7s
175:	learn: 0.9124239	total: 14.4s	remaining: 1m 7s
176:	learn: 0.9124428	total: 14.5s	remaining: 1m 7s
177:	learn: 0.9127211	total: 14.6s	remaining: 1m 7s
178:	learn: 0.91

318:	learn: 0.9177318	total: 26.9s	remaining: 57.5s
319:	learn: 0.9177472	total: 27s	remaining: 57.4s
320:	learn: 0.9177614	total: 27.1s	remaining: 57.3s
321:	learn: 0.9177875	total: 27.2s	remaining: 57.3s
322:	learn: 0.9178399	total: 27.3s	remaining: 57.2s
323:	learn: 0.9178482	total: 27.4s	remaining: 57.1s
324:	learn: 0.9178556	total: 27.5s	remaining: 57.1s
325:	learn: 0.9178738	total: 27.6s	remaining: 57s
326:	learn: 0.9178962	total: 27.6s	remaining: 56.9s
327:	learn: 0.9179020	total: 27.8s	remaining: 56.9s
328:	learn: 0.9179073	total: 27.8s	remaining: 56.8s
329:	learn: 0.9179108	total: 27.9s	remaining: 56.7s
330:	learn: 0.9179149	total: 28s	remaining: 56.6s
331:	learn: 0.9179237	total: 28.1s	remaining: 56.5s
332:	learn: 0.9179646	total: 28.2s	remaining: 56.5s
333:	learn: 0.9179705	total: 28.3s	remaining: 56.4s
334:	learn: 0.9179768	total: 28.4s	remaining: 56.3s
335:	learn: 0.9180110	total: 28.5s	remaining: 56.2s
336:	learn: 0.9180289	total: 28.5s	remaining: 56.2s
337:	learn: 0.9180

479:	learn: 0.9218012	total: 41s	remaining: 44.5s
480:	learn: 0.9218173	total: 41.1s	remaining: 44.4s
481:	learn: 0.9218366	total: 41.2s	remaining: 44.3s
482:	learn: 0.9218490	total: 41.3s	remaining: 44.2s
483:	learn: 0.9218636	total: 41.4s	remaining: 44.1s
484:	learn: 0.9218735	total: 41.4s	remaining: 44s
485:	learn: 0.9218985	total: 41.5s	remaining: 43.9s
486:	learn: 0.9219092	total: 41.6s	remaining: 43.8s
487:	learn: 0.9219127	total: 41.7s	remaining: 43.7s
488:	learn: 0.9219195	total: 41.8s	remaining: 43.7s
489:	learn: 0.9219210	total: 41.9s	remaining: 43.6s
490:	learn: 0.9219243	total: 41.9s	remaining: 43.5s
491:	learn: 0.9219422	total: 42s	remaining: 43.4s
492:	learn: 0.9219553	total: 42.1s	remaining: 43.3s
493:	learn: 0.9219653	total: 42.2s	remaining: 43.2s
494:	learn: 0.9219843	total: 42.3s	remaining: 43.2s
495:	learn: 0.9219921	total: 42.4s	remaining: 43.1s
496:	learn: 0.9219988	total: 42.5s	remaining: 43s
497:	learn: 0.9220175	total: 42.6s	remaining: 42.9s
498:	learn: 0.922038

638:	learn: 0.9236627	total: 54.7s	remaining: 30.9s
639:	learn: 0.9236925	total: 54.8s	remaining: 30.8s
640:	learn: 0.9236992	total: 54.8s	remaining: 30.7s
641:	learn: 0.9237263	total: 54.9s	remaining: 30.6s
642:	learn: 0.9237364	total: 55s	remaining: 30.5s
643:	learn: 0.9237516	total: 55.1s	remaining: 30.5s
644:	learn: 0.9238072	total: 55.2s	remaining: 30.4s
645:	learn: 0.9238087	total: 55.3s	remaining: 30.3s
646:	learn: 0.9238252	total: 55.4s	remaining: 30.2s
647:	learn: 0.9238440	total: 55.5s	remaining: 30.1s
648:	learn: 0.9238584	total: 55.5s	remaining: 30s
649:	learn: 0.9238787	total: 55.6s	remaining: 30s
650:	learn: 0.9239068	total: 55.7s	remaining: 29.9s
651:	learn: 0.9239244	total: 55.8s	remaining: 29.8s
652:	learn: 0.9239296	total: 55.9s	remaining: 29.7s
653:	learn: 0.9239392	total: 56s	remaining: 29.6s
654:	learn: 0.9239423	total: 56.1s	remaining: 29.5s
655:	learn: 0.9239768	total: 56.2s	remaining: 29.5s
656:	learn: 0.9239901	total: 56.3s	remaining: 29.4s
657:	learn: 0.923996

800:	learn: 0.9254186	total: 1m 9s	remaining: 17.3s
801:	learn: 0.9254228	total: 1m 9s	remaining: 17.2s
802:	learn: 0.9254282	total: 1m 9s	remaining: 17.1s
803:	learn: 0.9254323	total: 1m 9s	remaining: 17.1s
804:	learn: 0.9254389	total: 1m 10s	remaining: 17s
805:	learn: 0.9254442	total: 1m 10s	remaining: 16.9s
806:	learn: 0.9254475	total: 1m 10s	remaining: 16.8s
807:	learn: 0.9254552	total: 1m 10s	remaining: 16.7s
808:	learn: 0.9254578	total: 1m 10s	remaining: 16.6s
809:	learn: 0.9254629	total: 1m 10s	remaining: 16.6s
810:	learn: 0.9254760	total: 1m 10s	remaining: 16.5s
811:	learn: 0.9255023	total: 1m 10s	remaining: 16.4s
812:	learn: 0.9255108	total: 1m 10s	remaining: 16.3s
813:	learn: 0.9255133	total: 1m 10s	remaining: 16.2s
814:	learn: 0.9255168	total: 1m 11s	remaining: 16.1s
815:	learn: 0.9255253	total: 1m 11s	remaining: 16.1s
816:	learn: 0.9255285	total: 1m 11s	remaining: 16s
817:	learn: 0.9255444	total: 1m 11s	remaining: 15.9s
818:	learn: 0.9255523	total: 1m 11s	remaining: 15.8s
8

958:	learn: 0.9267204	total: 1m 24s	remaining: 3.6s
959:	learn: 0.9267247	total: 1m 24s	remaining: 3.51s
960:	learn: 0.9267265	total: 1m 24s	remaining: 3.42s
961:	learn: 0.9267343	total: 1m 24s	remaining: 3.34s
962:	learn: 0.9267375	total: 1m 24s	remaining: 3.25s
963:	learn: 0.9267432	total: 1m 24s	remaining: 3.16s
964:	learn: 0.9267541	total: 1m 24s	remaining: 3.07s
965:	learn: 0.9267595	total: 1m 24s	remaining: 2.98s
966:	learn: 0.9267652	total: 1m 24s	remaining: 2.9s
967:	learn: 0.9267671	total: 1m 24s	remaining: 2.81s
968:	learn: 0.9267760	total: 1m 25s	remaining: 2.72s
969:	learn: 0.9267791	total: 1m 25s	remaining: 2.63s
970:	learn: 0.9267807	total: 1m 25s	remaining: 2.55s
971:	learn: 0.9267829	total: 1m 25s	remaining: 2.46s
972:	learn: 0.9268001	total: 1m 25s	remaining: 2.37s
973:	learn: 0.9268132	total: 1m 25s	remaining: 2.28s
974:	learn: 0.9268143	total: 1m 25s	remaining: 2.19s
975:	learn: 0.9268220	total: 1m 25s	remaining: 2.11s
976:	learn: 0.9268238	total: 1m 25s	remaining: 2

In [5]:
preds = ctb.predict_proba(n_test)
res = pd.DataFrame({'Id':range(len(n_test)),'Prediction':preds[:,1]})
res.to_csv('submission.csv', sep=',',  index=False)

res

,Id,Prediction
0,0,0.298975
1,1,0.181919
2,2,0.182622
3,3,0.173703
4,4,0.301263
5,5,0.327527
6,6,0.805871
7,7,0.532827
8,8,0.532827
9,9,0.511435
